In [206]:
'''
File: scrape_reddit.ipynb
Purpose: CS 505 Final Project | Scraping reddit.com/r/baseball posts
Authors: Benjamin Owens, Jennifer Tsui
Last modification: November 11, 2016

Description:
This is a script that allows one to collect reddit posts from a subreddit (in this case, /r/baseball).
The script allows the user to collect data from any time period, specified by the user input. 

For the purposes of this assignment, we collected posts from the most recent full year (2015). The .csv
file derived from the full execution is included in the repository (data.csv), as well as the filtered data
that includes only posts including the mentions of teams and cities (posts_with_mentions.csv). 
The filtering component of the script is not finalized, but the methodology is sound 
(we'll need to include abbreviations, nicknames, common typos, etc. in the future). 

The runtime is on the order of 5-10 seconds per day, depending on the activity (number of posts).

The raw script is also included as a .py file.

'''

import praw
import pandas as pd
import numpy as np
import time
import datetime
import itertools as it


print("Enter date to start checking:\n (default 01/01/2015)")
print("")
start = ''
start = input("User input: ")
if start == '':
    start = "01/01/2015"

startStamp = int(time.mktime(datetime.datetime.strptime(start, "%m/%d/%Y").timetuple()))
print("Enter number of days to check: [>2] \n(Default 30)")
numDays = 0
numDays = input("User input: ")
if numDays == 0:
    numDays = "30"
days = [(int(startStamp)+ (86400*i)) for i in range(0,int(numDays))]

print("Requesting Data ")
r = praw.Reddit(user_agent='ben_and_jens_505_project')

'''
Here we use a helper function from the PRAW library that allows us to get the posts within a certain timestamp.
It seems to be asynchronous which is a little strange but it works!
'''
submissionList = []
# Get the first day's data
print("Checking day: " + str(datetime.datetime.fromtimestamp(int(days[0]))))
submission = praw.helpers.submissions_between(r,r.get_subreddit('baseball'),
                                                lowest_timestamp=days[0],highest_timestamp=days[1])


thelist = [str(x).split('::') + [str(datetime.datetime.fromtimestamp(int(days[0])))] for x in submission]
submissionList += thelist
# get the rest of the days' data
for i in range(1,len(days[1:])):
    print("Checking day: " + str(datetime.datetime.fromtimestamp(int(days[i]))))
    submission = praw.helpers.submissions_between(r,r.get_subreddit('baseball'),
                                                    lowest_timestamp=days[i],highest_timestamp=days[i+1],
                                                    verbosity=1)

    #split the string returned and add the date to every row
    thelist = [str(x).split('::') + [str(datetime.datetime.fromtimestamp(int(days[i])))] for x in submission]
    submissionList += thelist

scores = [x[0] for x in submissionList]
posts = [x[1] for x in submissionList]
dates = [x[2] for x in submissionList]
df = pd.DataFrame()
df["Score"] = scores
df["Title"] = posts
df["Date"] = dates

df.to_csv("data.csv")
print("Done writing 'data.csv'")


Enter date to start checking:
 (default 01/01/2015)



KeyboardInterrupt: 

In [201]:


# some preprocessing to filter out posts that dont mention a team or city
teams = ['Arizona',
 'Diamondbacks',
 'Atlanta',
 'Braves',
 'Baltimore',
 'Orioles',
 'Boston',
 'Red',
 'Sox',
 'Chicago',
 'Cubs',
 'Chicago',
 'White',
 'Sox',
 'Cincinnati',
 'Reds',
 'Cleveland',
 'Indians',
 'Colorado',
 'Rockies',
 'Detroit',
 'Tigers',
 'Miami',
 'Marlins',
 'Houston',
 'Astros',
 'Kansas',
 'City',
 'Royals',
 'Los',
 'Angeles',
 'Angels',
 'Anaheim',
 'Los',
 'Angeles',
 'Dodgers',
 'Milwaukee',
 'Brewers',
 'Minnesota',
 'Twins',
 'New',
 'York',
 'Mets',
 'New',
 'York',
 'Yankees',
 'Oakland',
 'Athletics',
 'Philadelphia',
 'Phillies',
 'Pittsburgh',
 'Pirates',
 'Saint',
 'Louis',
 'Cardinals',
 'San',
 'Diego',
 'Padres',
 'San',
 'Francisco',
 'Giants',
 'Seattle',
 'Mariners',
 'Tampa',
 'Bay',
 'Rays',
 'Texas',
 'Rangers',
 'Toronto',
 'Blue',
 'Jays',
 'Washington',
 'Nationals']
df = pd.read_csv('data.csv')
filtered = df[df['Title'].str.split().apply(lambda x: len(set(x).intersection(set(teams))))>0]
filtered.to_csv("posts_with_mentions.csv")          
print("Done writing 'posts_with_mentions.csv'")

#EOF

Done writing 'posts_with_mentions.csv'


In [202]:
# TODO: count the mention of each team
df = pd.read_csv('posts_with_mentions.csv')
df = df.drop("Unnamed: 0",1)
teams = ['ANA', 'ARI', 'ATL', 'BAL', 'BOS', 'CWS', 'CHC', 'CIN', 'CLE', 'COL', 'DET', 'HOU', 'KCR', 'LAD', 'MIA', 'MIL', 'MIN', 'NYY', 'NYM', 'OAK', 'PHI', 'PIT', 'SDP', 'SEA', 'SFG', 'SLC', 'TBR', 'TEX', 'TOR', 'WAS']
teamDict = dict.fromkeys(teams)
zeroes = pd.DataFrame(0,columns=teams,index=df.index)
df = df.join(zeroes)

teamDict = {'ANA':["Anaheim","Angels",], 
 'ARI':["Arizona",'ARI','Diamondbacks','Chase Field'], 
 'ATL':["Atlanta",'ATL','Braves','Suntrust Park'], 
 'BAL':["Baltimore","Orioles",' BAL ','Camden Yards'], 
 'BOS':["Boston","Red","Red Sox",'BOS','Fenway','boston'], 
 'CWS':['White Sox','Guaranteed Rate'], 
 'CHC':["Cubs","Wrigley"], 
 'CIN':["Cincinatti",'Reds','Great American'], 
 'CLE':["Cleveland","Indians",'Progressive'], 
 'COL':["Colorado",'Rockies',"Coors"], 
 'DET':["Detriot","Tigers",'comerica'], 
 'HOU':["Houston","Astros",'minute maid'], 
 'KCR':["Kansas","Kansas City","Royals","KC",'kauffman'], 
 'LAD':["Angeles","LA ", "Dodgers"], 
 'MIA':["Miami",'Marlins'], 
 'MIL':["Milwaukee","Brewers","Miller Park"], 
 'MIN':["Minnesota","Twins","Target Field"], 
 'NYY':["Yankees","Yankee","Bronx"], 
 'NYM':["Mets","Citi","Queens"], 
 'OAK':['Athletics','Coliseum'], 
 'PHI':["Pittsburg","Pirates","PNC"], 
 'PIT':["Philadelphia","Phillies"], 
 'SDP':["Diego","Padres","Petco"], 
 'SEA':["Seattle","Mariners","Safeco"], 
 'SFG':["Francisco","giants"], 
 'SLC':["St","Louis","Saint","Cardinals","Busch"], 
 'TBR':["Tampa","Rays","Tropicana"], 
 'TEX':["Texas","Rangers","Globe Life","Arlington"], 
 'TOR':["Toronto","Blue Jays","Jays","rogers centre"], 
 'WAS':["Washington","Nationals"]}


# Initialize date variables
first_date = pd.to_datetime(df.iloc[0]['Date'])
last_date = pd.to_datetime(df.iloc[-1]['Date'])
week_delta = datetime.timedelta(days=7)
day_delta = datetime.timedelta(days=1)
hour_delta = datetime.timedelta(hours=1)
week_starts = [first_date]

# Create keys as the first day of every week
week_starts[-1] = pd.to_datetime(week_starts[-1])
# Create an array of all of the first days of the weeks
while(week_starts[-1] < last_date):
    week_starts.append(week_starts[-1] + week_delta)    
# create a dictionary from the array
week_dict = {key: None for key in week_starts}

# Remove DST
df['Date'] = df['Date'].apply(lambda date: pd.datetime.strptime(date,"%Y-%m-%d %H:%M:%S").replace(hour=0))







In [205]:
# Count mentions by week
for week in week_dict.keys():
    #print(week)
    week_performance = dict.fromkeys(teams)
    week = pd.to_datetime(week)
    #For every day this week
    for i in range(0,7):
        date = week + (day_delta*i)
        date = date.replace(hour=0)
        #print(date)
        date_with_dst = [str(date), str(date + hour_delta)]
        #print(date)
        # For every post on this day
        
        for post in df.loc[df['Date'] == date].iterrows():
            #print(post[1]['Title'])
            # For every team
            for team in teamDict.keys():
                #Initialize dict values to an int
                if week_performance[str(team)] == None:
                        week_performance[str(team)] = 0
                #print(team)
                # Set is team in flag
                is_team_in = False
                # for every word associated with that team
                for word in teamDict[team]:
                    #print(word)
                    # if that word is in the post title
                    if word in post[1]['Title']:
                        # set the flag to true and stop checking
                        is_team_in = True
                        break
                        
                if is_team_in == True:              
                    #print("Found: " + team)
                    # DEPRECIATED: here is where we "weight" posts. Higher scored posts get more weight
                    # CURRENT: what we do here is simple add the post scores
                    if post[1]['Score'] > 2000:
                        week_performance[str(team)] += 20
                    elif post[1]['Score'] > 1000:
                        week_performance[str(team)] += 10
                    elif post[1]['Score'] > 100:
                        week_performance[str(team)] += 5
                    elif post[1]['Score'] > 10:
                        week_performance[str(team)] += 2
                    else: week_performance[str(team)] += 1
                    #post[1]['Score']
    
        
                
               
               
    week_dict[week] = week_performance

# Convert dict to pandas

results = pd.DataFrame(week_dict)
results.to_csv("reddit_stats.csv")
#results
df_1_columns = list(results.columns.values)
df_1 = results[df_1_columns]
df_1




,2015-01-01 00:00:00,2015-01-08 00:00:00,2015-01-15 00:00:00,2015-01-22 00:00:00,2015-01-29 00:00:00,2015-02-05 00:00:00,2015-02-12 00:00:00,2015-02-19 00:00:00,2015-02-26 00:00:00,2015-03-05 00:00:00,...,2015-10-29 00:00:00,2015-11-05 00:00:00,2015-11-12 00:00:00,2015-11-19 00:00:00,2015-11-26 00:00:00,2015-12-03 00:00:00,2015-12-10 00:00:00,2015-12-17 00:00:00,2015-12-24 00:00:00,2015-12-31 00:00:00
ANA,8,2,1,0,3,2,33,0,5,0,...,2,2,24,5,3,7,23,5,0,None
ARI,7,7,0,0,2,3,0,5,1,3,...,0,19,4,10,11,34,14,7,0,None
ATL,3,7,35,8,3,2,7,4,0,8,...,0,9,30,16,14,36,15,7,2,None
BAL,4,7,1,7,0,7,3,71,3,7,...,0,12,2,7,10,10,27,7,7,None
BOS,14,9,4,11,11,16,11,27,1,11,...,10,12,14,10,41,49,26,19,22,None
CHC,2,8,14,12,7,2,7,12,14,15,...,24,6,9,7,10,59,45,9,2,None
CIN,0,0,0,2,6,8,0,8,0,8,...,6,12,3,2,4,12,9,10,6,None
CLE,0,5,24,4,2,0,7,2,5,12,...,1,3,5,2,16,5,1,10,0,None
COL,2,5,2,0,4,5,0,0,0,2,...,2,17,1,6,0,10,10,12,0,None
CWS,5,3,5,3,12,5,7,2,7,5,...,7,0,0,0,2,11,4,2,8,None
